I want to play around with Talos. Let's begin.

https://www.talos.dev/v1.7/introduction/quickstart/

In [2]:
%%sh
nix profile install nixpkgs#talosctl 

this path will be fetched (23.07 MiB download, 102.63 MiB unpacked):
  /nix/store/73cfwlcil9xicnqm0qvrgh4aa4adcav2-talosctl-1.7.4
copying path '/nix/store/73cfwlcil9xicnqm0qvrgh4aa4adcav2-talosctl-1.7.4' from 'https://cache.nixos.org'...


In [5]:
%%sh
talosctl cluster create

validating CIDR and reserving IPs
generating PKI and tokens
permission denied while trying to connect to the Docker daemon socket at unix:///var/run/docker.sock: Head "http://%2Fvar%2Frun%2Fdocker.sock/_ping": dial unix /var/run/docker.sock: connect: permission denied


CalledProcessError: Command 'b'talosctl cluster create\n'' returned non-zero exit status 1.

Looks like Talos is trying to connect to the rootful Docker daemon. I have rootless docker installed.

In [6]:
%env DOCKER_HOST=unix:///run/user/1000/docker.sock

env: DOCKER_HOST=unix:///run/user/1000/docker.sock


In [8]:
%%sh
talosctl cluster create 

validating CIDR and reserving IPs
generating PKI and tokens
creating network talos-default
creating controlplane nodes
1 error occurred:
	* Error response from daemon: Conflict. The container name "/talos-default-controlplane-1" is already in use by container "fcb9609ed4e4dbea120cf792b4d37d89b86ca81618fde3846dc3088297eb2ecc". You have to remove (or rename) that container to be able to reuse that name.




CalledProcessError: Command 'b'talosctl cluster create \n'' returned non-zero exit status 1.

Talos ended up waiting forever for the API to be ready. I think this is because I tried to install it on my rootless Docker install. I'm going to try again with a rootful Docker Daemon.

In [18]:
%%sh
talosctl cluster destroy

destroying node talos-default-controlplane-1
destroying node talos-default-worker-1
destroying network talos-default


In [13]:
%env DOCKER_HOST=unix:///var/run/docker.sock

env: DOCKER_HOST=unix:///var/run/docker.sock


In [16]:
%%sh
docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [26]:
%%sh
talosctl cluster create

validating CIDR and reserving IPs
generating PKI and tokens
creating network talos-default
creating controlplane nodes
creating worker nodes
renamed talosconfig context "talos-default" -> "talos-default-2"


waiting for API
bootstrapping cluster


waiting for etcd to be healthy: ...
waiting for etcd to be healthy: 1 error occurred:
    * 10.5.0.2: service "etcd" not in expected state "Running": current state [Preparing] Running pre state
waiting for etcd to be healthy: 1 error occurred:
    * 10.5.0.2: service is not healthy: etcd
waiting for etcd to be healthy: OK
waiting for etcd members to be consistent across nodes: ...
waiting for etcd members to be consistent across nodes: OK
waiting for etcd members to be control plane nodes: ...
waiting for etcd members to be control plane nodes: OK
waiting for apid to be ready: ...
waiting for apid to be ready: OK
waiting for all nodes memory sizes: ...
waiting for all nodes memory sizes: OK
waiting for all nodes disk sizes: ...
waiting for all nodes disk sizes: OK
waiting for kubelet to be healthy: ...
waiting for kubelet to be healthy: 1 error occurred:
    * 10.5.0.2: service "kubelet" not in expected state "Running": current state [Preparing] Running pre state
waiting for kubelet to

renamed cluster "talos-default" -> "talos-default-1"
renamed auth info "admin@talos-default" -> "admin@talos-default-1"
renamed context "admin@talos-default" -> "admin@talos-default-1"
PROVISIONER           docker
NAME                  talos-default
NETWORK NAME          talos-default
NETWORK CIDR          10.5.0.0/24
NETWORK GATEWAY       10.5.0.1
NETWORK MTU           1500
KUBERNETES ENDPOINT   https://127.0.0.1:39773

NODES:

NAME                            TYPE           IP         CPU    RAM      DISK
/talos-default-controlplane-1   controlplane   10.5.0.2   2.00   2.1 GB   -
/talos-default-worker-1         worker         10.5.0.3   2.00   2.1 GB   -


In [32]:
%%sh
talosctl stats --nodes 10.5.0.2

NODE       NAMESPACE   ID       MEMORY(MB)   CPU
10.5.0.2   system      apid     22.58        439572000
10.5.0.2   system      trustd   15.05        128127000


In [63]:
%%sh
docker ps

CONTAINER ID   IMAGE                             COMMAND        CREATED          STATUS          PORTS                                                   NAMES
1bebd03746cc   ghcr.io/siderolabs/talos:v1.7.4   "/sbin/init"   22 minutes ago   Up 22 minutes                                                           talos-default-worker-1
59f7d130c0bf   ghcr.io/siderolabs/talos:v1.7.4   "/sbin/init"   22 minutes ago   Up 22 minutes   127.0.0.1:39773->6443/tcp, 127.0.0.1:35903->50000/tcp   talos-default-controlplane-1


`talosctl dashboard --nodes 10.5.0.2`
![a view of the talosctl dashboard](talosctl_dashboard.png)

In [35]:

%%sh
kubectl get nodes -o wide

NAME                           STATUS   ROLES           AGE     VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE         KERNEL-VERSION   CONTAINER-RUNTIME
talos-default-controlplane-1   Ready    control-plane   8m28s   v1.30.1   10.5.0.2      <none>        Talos (v1.7.4)   6.1.0-21-amd64   containerd://1.7.16
talos-default-worker-1         Ready    <none>          8m26s   v1.30.1   10.5.0.3      <none>        Talos (v1.7.4)   6.1.0-21-amd64   containerd://1.7.16


In [55]:
%%sh
cat > pod.yaml << EOF
apiVersion: v1
kind: Pod
metadata:
    name: searxng
spec:
    containers:
      - name: searxng
        image: searxng/searxng
        ports:
        - containerPort: 8080
          hostPort: 8080
EOF

kubectl apply -f pod.yaml

Error from server (Forbidden): error when creating "pod.yaml": pods "searxng" is forbidden: violates PodSecurity "baseline:latest": hostPort (container "searxng" uses hostPort 8080)


CalledProcessError: Command 'b'cat > pod.yaml << EOF\napiVersion: v1\nkind: Pod\nmetadata:\n    name: searxng\nspec:\n    containers:\n      - name: searxng\n        image: searxng/searxng\n        ports:\n        - containerPort: 8080\n          hostPort: 8080\nEOF\n\nkubectl apply -f pod.yaml\n'' returned non-zero exit status 1.

Looks like Talos doesn't like the `hostPort`.

In [60]:
%%sh
cat > pod.yaml << EOF
apiVersion: v1
kind: Pod
metadata:
    name: searxng
spec:
    containers:
      - name: searxng
        image: searxng/searxng
        ports:
        - containerPort: 8080
EOF

kubectl apply -f pod.yaml

pod/searxng created


This time we only got a warning. Did the pod get created?

In [58]:
%%sh
kubectl get pods 

NAME      READY   STATUS    RESTARTS   AGE
searxng   1/1     Running   0          2s


Looks like it was. Can we access it?

In [64]:
%%sh
curl localhost:8080

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to localhost port 8080 after 0 ms: Couldn't connect to server


CalledProcessError: Command 'b'curl localhost:8080\n'' returned non-zero exit status 7.

Nope! I think I will continue this another day, once I learn more about Kubernetes. Until them, let's clean up.

In [59]:
%%sh
kubectl delete pod searxng

pod "searxng" deleted


In [65]:
%%sh
talosctl cluster destroy

destroying node talos-default-controlplane-1
destroying node talos-default-worker-1
destroying network talos-default
